In [57]:
# initial imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import hvplot.pandas
import panel as pn
from pathlib import Path
from dotenv import load_dotenv
from panel.interact import interact
from panel import widgets
from string import digits

pn.extension()

%matplotlib inline

In [58]:
file_path = Path("Resources/Raw/ATLAS.csv")
ATLAS_raw = pd.read_csv(file_path, index_col="Customers")
#ATLAS_raw.value_counts()
ATLAS_raw.describe()

,Invoice Date,Invoice #,Invoice Amount,Subscription,Account Code,Dates of service
count,669,665,669,669,669,669
unique,317,654,212,8,8,409
top,9/8/2020,AJ515,$750.00,1 Year,4700-0-00-00000-00-0000,1 Year Subscription 8/31/2020 to 8/31/2021
freq,29,4,60,431,661,18


In [59]:
# Read the ATLAS data into a Pandas DataFrame
file_path = Path("Resources/Raw/ATLAS.csv")
ATLAS_raw = pd.read_csv(file_path, index_col="Customers")
ATLAS_raw.head()
ATLAS_raw[0:10]


# Function definitions
# TODO move to .py
def remove_strings_from_customer_names(original_customer_name):
    invalid_strings = ["pymt", "due"]
    for invalid_item in invalid_strings:
        original_customer_name = original_customer_name.replace(invalid_item, "")
    return original_customer_name.strip()

def remove_numbers_from_customer_names(original_customer_name):
    remove_digits = str.maketrans('', '', digits) 
    return original_customer_name.translate(remove_digits).strip()  


# Extract paying customers
paying_customers_raw = ATLAS_raw.index


# Clean up names
paying_customers_cleanedup = []
for customer in paying_customers_raw:
    corrected_customer_name = remove_strings_from_customer_names(customer)
    corrected_customer_name = remove_numbers_from_customer_names(corrected_customer_name)
    paying_customers_cleanedup.append(corrected_customer_name)
    #print(f"Raw customer name: \"{customer}\"  Corrected customer name: \"{corrected_customer_name}\"")


# Define needed variables for re-naming
name_mapping = {}
n = 1


# Build name mapping
for customer in paying_customers_cleanedup:
    if not customer in name_mapping:
        name_mapping[customer] = "University " + str(n)
        n += 1


# Re-name the universities in the data
paying_customers_deanonymized = []
for customer in paying_customers_cleanedup:
    paying_customers_deanonymized.append(name_mapping[customer])


# Build a de-anonymized DataFrame
ATLAS_deanon = pd.concat([
    ATLAS_raw["Invoice Date"],
    ATLAS_raw["Invoice #"],
    ATLAS_raw["Invoice Amount"],
    ATLAS_raw["Subscription"],
    ATLAS_raw["Account Code "],
    ATLAS_raw["Dates of service "]
    ], axis="columns", join="inner")

## Fix index
ATLAS_deanon.index = paying_customers_deanonymized
ATLAS_deanon.index.name = "Customers"

## Result
ATLAS_deanon.head()


# Export
ATLAS_deanon.to_csv("Resources/RawDeanonymized/ATLAS.csv")